<div style="width: 100%; background-color: #ef7d22; text-align: center">
<br><br>

<h1 style="color: white; font-weight: bold;">
    PostgreSQL Data Definition Language
</h1>

<br><br> 
</div>

Various RDBMSs implement languages close to official SQL specifications, but rarely either a strict superset nor a strict subset of actual specs.  It is complicated further by the fact that SQL specs include SQL-86, SQL-89, SQL-92, SQL:1999, SQL:2003, SQL:2006, SQL:2008, SQL:2011, and SQL:2016.  Each of these are different, not only by adding features but also sometimes deprecating them or changing spelling.

In other words, while most data definition commands you will use in PostgreSQL are mostly similar to the equivalent commands in other RDBMSs, the equivalence is not precise.  PostgreSQL supports a large number of data definition commands (all SQL-like); this lesson will highlight some particularly commonly used ones.

## CREATE, DROP, ALTER
![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

The basic verbs for working with *data* in PostgreSQL are SELECT, INSERT, UPDATE, DELETE, and perhaps TRUNCATE.  The first of those does not actually change data, just retrieve it.  But the verbs of the PostgreSQL data definitions language (DDL) are CREATE, DROP, and ALTER.  GRANT is borderline, but possibly worth including as DDL.  These affect the *shape* of data rather than the content as such; however, many changes to shape by implication change underlying data globally (such as by removing it, or by changing its data type).

Each regular DDL verb is followed by a noun, and usually by some additional qualifiers.  For example, we can DROP the following objects:

|     Objects   |  Objects      |  Objects
|:--------------|:--------------|:--------------
| ACCESS METHOD | AGGREGATE     | CAST
| COLLATION     | CONVERSION    | DATABASE
| DOMAIN        | EVENT TRIGGER | EXTENSION
| FOREIGN TABLE | FUNCTION      | GROUP
| INDEX         | LANGUAGE      | MATERIALIZED VIEW
| OPERATOR      | OPERATOR CLASS| OPERATOR FAMILY
| OWNED         | POLICY        | PROCEDURE
| PUBLICATION   | ROLE          | ROUTINE
| RULE          | SCHEMA        | SEQUENCE
| SERVER        | STATISTICS    | SUBSCRIPTION
| TABLE         | TABLESPACE    | TEXT SEARCH CONFIGURATION
| TEXT SEARCH DICTIONARY | TEXT SEARCH PARSER | TEXT SEARCH TEMPLATE
| TRANSFORM     | TRIGGER       | TYPE
| USER          | USER MAPPING  | VIEW

Most of those same things can be CREATE'd or ALTER'd.  All of these are useful, of course, but USER/ROLE, TABLE, DATABASE, and INDEX are those you will use during typical database configuration.

Although the steps in this lesson are ones you would typically perform in an administration interface, let us work within the notebook, and using a `psycopg2` connection.

In [1]:
import psycopg2
user = 'ine_student'
pwd = 'ine-password'
host = 'localhost'
port = '5432'
db = 'ine'

conn = psycopg2.connect(database=db, host=host, user=user, password=pwd, port=port)
cur = conn.cursor()

## Refactoring tables
![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

Let us recall the definition of the `census_zipcode_geography` table.  It has some flaws we can improve.  We might define a simple function to view table definitions from Python.

In [2]:
def table_schema(table_name, table_schema=None, conn=conn):
    try:
        cur = conn.cursor()
        if table_schema is None:
            sql = """
            SELECT column_name, data_type, character_maximum_length, 
                   column_default, is_nullable
            FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = %(name)s;
            """
        else:
            sql = """
            SELECT column_name, data_type, character_maximum_length, 
                   column_default, is_nullable
            FROM INFORMATION_SCHEMA.COLUMNS 
            WHERE table_name = %(name)s AND table_schema = %(schema)s;
            """
        cur.execute(sql, {'name': table_name, 'schema': table_schema})
        return cur.fetchall()
    except Exception as err:
        print("Problem encountered:", err)
        conn.rollback()

Viewing the table, we can see significant redundancy.  `awater` and `aland` express the same quantity as `awater_sqmi` and `aland_sqmi`, respectively, only in different units (square meters versus square miles).  Moreover, `location` is simply a combination of `intptlat` and `intptlong`.  Not only does this waste storage space, it could potentially lead to inconsistencies in the data.

In [3]:
table_schema('census_zipcode_geography')

[('usps', 'character', 5, None, 'NO'),
 ('aland', 'bigint', None, None, 'YES'),
 ('awater', 'bigint', None, None, 'YES'),
 ('aland_sqmi', 'numeric', None, None, 'YES'),
 ('awater_sqmi', 'numeric', None, None, 'YES'),
 ('intptlat', 'real', None, None, 'YES'),
 ('intptlong', 'real', None, None, 'YES'),
 ('location', 'point', None, None, 'YES')]

## Schema namespaces
![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

Another problem suggests itself also.  In the names of some tables, we used an implicit namespace by separating components with underscores.  PostgreSQL offers us a canonical way to do this instead, with actual namespacing and isolation.  

For example, we might create both a table `home.users` and a table `business.users` that would vary not only in have distinct names, but also would have different functions, stored user FUNCTIONs, stored PROCEDUREs, OPERATORs, and other things pertaining to that namespace.

In [4]:
# Dropping normally rare, only for the lesson
try:
    cur.execute('DROP SCHEMA census CASCADE;')
except Exception as err:
    conn.rollback()
    print("Rolling back:", repr(err), sep='\n')
else:
    conn.commit()

In [5]:
cur.execute('CREATE SCHEMA census;')
conn.commit()

The table `census_zipcode_fields` has a simple design we need not improve.  Let us copy it in the new namespace (potentially we will eventually delete the tables with suboptimal design).

In [6]:
cur.execute('CREATE TABLE census.zipcode_fields AS TABLE census_zipcode_fields;')
conn.commit()

In [7]:
cur.execute('SELECT * FROM census.zipcode_fields')
cur.fetchmany(3)

[('USPS', 'United States Postal Service State Abbreviation'),
 ('GEOID',
  'Geographic Identifier - fully concatenated geographic code (State FIPS and district number)'),
 ('ALAND',
  'Land Area (square meters) - Created for statistical purposes only')]

Similarly, perhaps we should namespace the `users`, but here we want to modify the table somewhat as well to accomodate a zipcode field.  Zipcodes should exist in foreign table describing them. Let us first create that reference table.

In [8]:
cur.execute("CREATE TABLE census.zipcodes (zipcode CHAR(5) PRIMARY KEY);")
zips = "INSERT INTO census.zipcodes SELECT usps FROM census_zipcode_geography;"
cur.execute(zips)
conn.commit()

And now the schema and table for business users.

In [9]:
cur.execute("DROP SCHEMA IF EXISTS business CASCADE;")
cur.execute("CREATE SCHEMA business;")

biz_users = """
CREATE TABLE business.users (
  user_id SERIAL PRIMARY KEY,
  username VARCHAR(50) UNIQUE NOT NULL,
  password VARCHAR(50) NOT NULL,
  age SMALLINT,
  created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
  zipcode CHAR(5) DEFAULT NULL REFERENCES census.zipcodes(zipcode) ON DELETE SET NULL
);"""
cur.execute(biz_users)

Copy over the relevant data from the non-namespaced version.

In [10]:
sql_copy_users = """
INSERT INTO business.users (username, password, age)
SELECT username, password, age FROM users;"""
try:
    cur.execute(sql_copy_users)
except Exception as err:
    conn.rollback()
    print("Rolling back:", repr(err), sep='\n')
else:
    conn.commit()

In [11]:
from pprint import pprint
cur.execute("SELECT * FROM business.users LIMIT 2;")
for row in cur:
    pprint(row)

(1,
 'Alice',
 'bad_pw_1',
 37,
 datetime.datetime(2020, 12, 1, 19, 25, 1, 203001, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-300, name=None)),
 None)
(2,
 'Bob',
 'bad_pw_2',
 None,
 datetime.datetime(2020, 12, 1, 19, 25, 1, 203001, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-300, name=None)),
 None)


Let us test the foreign key constraint we added.

In [12]:
wanderer = ('Wilma', 'whatever', None, "ether")
try:
    sql = """
      INSERT INTO business.users (username, password, age, zipcode)
      VALUES (%s, %s, %s, %s)"""
    cur.execute(sql, wanderer)
except Exception as err:
    conn.rollback()
    print("Rolling back:", repr(err), sep='\n')
else:
    conn.commit()

Rolling back:
ForeignKeyViolation('insert or update on table "users" violates foreign key constraint "users_zipcode_fkey"\nDETAIL:  Key (zipcode)=(ether) is not present in table "zipcodes".\n')


## Generated columns
![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

Suppose we with to emulate initial `census_zipcode_geography` table, but enforce data integrity among columns.  We might define a new table as:

In [13]:
sql_geography = """
CREATE TABLE census.zipcode_geography (
  zipcode CHAR(5) PRIMARY KEY,   -- by implication, UNIQUE NOT NULL
  usps VARCHAR GENERATED ALWAYS AS (zipcode) STORED, 
                              -- actually physically stored for now
  aland BIGINT,               -- m^2 
  awater BIGINT,
  aland_sqmi NUMERIC(8, 3) GENERATED ALWAYS AS (aland / 2589988.1103) STORED, 
                              -- converted mi^2 from m^2
  awater_sqmi NUMERIC(8, 3) GENERATED ALWAYS AS (awater / 2589988.1103) STORED,
  intptlat REAL GENERATED ALWAYS AS (location[0]) STORED,
  intptlong REAL GENERATED ALWAYS AS (location[1]) STORED,
  location POINT              -- use geometric type for lat/lon
);
"""
cur.execute(sql_geography)
conn.commit()

As of PostgreSQL 12, generated columns are always generated on write operations and explicitly stored.  In future versions, an option to generate them only on read might become available.  We will see VIEWs in a later lesson, which can accomplish a similar purpose.

Let us insert an invented data row to demonstrate.

In [14]:
sql_99999 = """
INSERT INTO census.zipcode_geography (zipcode, aland, awater, location)
VALUES (99999, 25000000, 10000000, '(37.1, -95.7)')
"""
cur.execute(sql_99999)
cur.execute("SELECT * FROM census.zipcode_geography WHERE zipcode='99999';")
cols = (c.name for c in cur.description)
for row in cur:
    pprint(dict(zip(cols, row)))

# Don't actually keep this data
conn.rollback()

{'aland': 25000000,
 'aland_sqmi': Decimal('9.653'),
 'awater': 10000000,
 'awater_sqmi': Decimal('3.861'),
 'intptlat': 37.1,
 'intptlong': -95.7,
 'location': '(37.1,-95.7)',
 'usps': '99999',
 'zipcode': '99999'}


We can copy over the non-generated and non-namespaced data into the new table.

In [15]:
sql_copy_zips = """
INSERT INTO census.zipcode_geography (zipcode, aland, awater, location)
SELECT usps, aland, awater, location FROM census_zipcode_geography;
"""
cur.execute(sql_copy_zips)
conn.commit()

The census bureau provides the data as both square meters and square miles.  It appears that in 123 of the 33,144 rows, their conversion was not identical to ours.  Whether this matters is a domain judgement.

In [16]:
cur.execute("""
  SELECT new.zipcode, new.aland_sqmi as converted, old.aland_sqmi reported
  FROM census_zipcode_geography old
  JOIN census.zipcode_geography new
  ON (old.usps = new.zipcode)
  WHERE new.aland_sqmi != old.aland_sqmi;""")
cur.fetchmany(10)

[('00685', Decimal('77.532'), Decimal('77.533')),
 ('03741', Decimal('76.932'), Decimal('76.933')),
 ('06074', Decimal('28.055'), Decimal('28.056')),
 ('13656', Decimal('85.074'), Decimal('85.075')),
 ('14454', Decimal('50.623'), Decimal('50.624')),
 ('20106', Decimal('60.225'), Decimal('60.226')),
 ('24018', Decimal('45.385'), Decimal('45.386')),
 ('24426', Decimal('332.864'), Decimal('332.865')),
 ('26298', Decimal('11.096'), Decimal('11.097')),
 ('28504', Decimal('123.768'), Decimal('123.769'))]

## Altering tables
![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

We have copied data from some tables into related tables to refactor the data organization.  We can also change tables in place.  This should be done with caution, since some such alterations will lose information.  In many cases, we can make alterations such as changing to a compatible data type or adding columns, without harming existing data.

Recall the simple table that records long descriptions of zipcode field keys:

In [17]:
table_schema("zipcode_fields", table_schema="census")

[('key', 'character varying', 15, None, 'YES'),
 ('description', 'character varying', None, None, 'YES')]

In [18]:
sql_fixkey = """
ALTER TABLE census.zipcode_fields
ALTER key TYPE CHAR(20);  -- using fixed length keys
"""
cur.execute(sql_fixkey)

In [19]:
sql_refcol = """
ALTER TABLE census.zipcode_fields
ADD reference VARCHAR DEFAULT 'https://www.census.gov/data/tables.html';
"""
cur.execute(sql_refcol)

In [20]:
cur.execute('SELECT * FROM census.zipcode_fields LIMIT 3;')
cur.fetchall()

[('USPS                ',
  'United States Postal Service State Abbreviation',
  'https://www.census.gov/data/tables.html'),
 ('GEOID               ',
  'Geographic Identifier - fully concatenated geographic code (State FIPS and district number)',
  'https://www.census.gov/data/tables.html'),
 ('ALAND               ',
  'Land Area (square meters) - Created for statistical purposes only',
  'https://www.census.gov/data/tables.html')]

In [21]:
conn.commit()

## Summary
![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

We looked here at highlights of the PostgreSQL DDL.  Various other elements of it are shown in other lessons, both before and after this one.

In the next lesson we look at data types available for PostgreSQL.